In [6]:
import sys
sys.path.append("/home/ubuntu/anaconda2/lib/python2.7/site-packages")


In [7]:
WORD_EMBEDDING_VECTOR_PATH = 'GoogleNews-vectors-negative300.bin'
DUMPED_VECTOR_DIR = '/vectors_mb_new/'


In [8]:
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import gensim.models.keyedvectors as word2vec
import codecs
import pandas as pd
from keras.preprocessing import sequence


In [9]:
MAX_NB_WORDS = 5000


In [5]:
def get_vocabulary_size(data):
    words = set()

    for doc in data:
        tokens = doc.split()
        for t in tokens:
            words.add(t)

    print('\n'.join(sorted(words)))
    print(len(words))

In [6]:
def loadGloveModel(gloveFile):
    print("Loading Glove Model")
    f = codecs.open(gloveFile, 'r', encoding='latin-1').read().split('\n')
    model = {}
    for line in f:
        splitLine = line.split(' ')
        word = splitLine[0]
        # print(word)
        embedding = [float(val) for val in splitLine[1:]]
        model[word] = embedding
    print("Done.", len(model), " words loaded!")
    return model

In [15]:
def convert_into_sequences(df, colname):
    tokenizer = Tokenizer(nb_words=MAX_NB_WORDS)
    #if(df[colname].values) is str:
    tokenizer.fit_on_texts(str(df[colname].values))
    sequences = tokenizer.texts_to_sequences(str(df[colname].values))
    
    sl = max(list(map(lambda x: len(x), sequences)))
    print(sl)
    # print(df[colname].values[:10])
    print(sequences[:33])
    print(max(tokenizer.word_index.values()), len(tokenizer.word_index.values()))

    lens = list(map(lambda s: len(s), sequences))
    print(len(lens), max(lens))
    data = pad_sequences(sequences, maxlen=sl)
    print(data[:10])

  
    index_to_word = {v: k for k, v in tokenizer.word_index.items()}
  
    # Load word embeddings and create embedding matrix
    print('Loading Embeddings')
    model = word2vec.KeyedVectors.load_word2vec_format(WORD_EMBEDDING_VECTOR_PATH, binary=True, encoding='utf-8')
    # model = loadGloveModel(config.WORD_EMBEDDING_VECTOR_PATH)

    print('Loaded')
    print(model['try'])
    #
    nf = set()
    embedding_matrix = np.zeros((len(tokenizer.word_index) + 1, 300))
    counter = 0
    for word, i in tokenizer.word_index.items():

        embedding_vector = np.zeros(300)
        if word in model:
            embedding_matrix[i] = model[word]
            counter += 1
        else:
            nf.add(word)
            embedding_matrix[i] = embedding_vector

    print(len(nf))
    print(len(data))
    print(len(embedding_matrix))
    joblib.dump(embedding_matrix,'mb_voc_embeddings.pkl')
    joblib.dump(data,'mb_sequences.pkl')

    print(len(data))
    print(len(embedding_matrix))
    print(data.shape)
    print(embedding_matrix.shape)
    print(counter)

    print(nf)
    return None, None


In [11]:
df = pd.read_csv('mb_train_trial_test_new_raw.csv')


In [10]:
from keras.preprocessing import sequence
from sklearn.externals import joblib



In [18]:
convert_into_sequences(df, 'text')

/usr/local/lib/python3.5/dist-packages/Keras-2.0.8-py3.5.egg/keras/preprocessing/text.py:145: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.


1
[[], [1], [4], [1], [], [1], [4], [1], [], [1], [4], [1], [], [], [], [], [], [], [], [], [], [9], [14], [10], [1], [], [8], [5], [8], [2], [11], [6], [5]]
28 28
206 1
[[0]
 [1]
 [4]
 [1]
 [0]
 [1]
 [4]
 [1]
 [0]
 [1]]
Loading Embeddings
Loaded
[  2.40234375e-01   2.01171875e-01   1.62109375e-01   2.08984375e-01
   1.15966797e-02  -2.58789062e-02   2.33398438e-01  -3.56445312e-02
  -1.05957031e-01  -1.47460938e-01  -1.02539062e-02   1.03149414e-02
  -6.17675781e-02  -2.79541016e-02  -2.08007812e-01  -4.12597656e-02
   7.81250000e-02   2.53906250e-01   2.32421875e-01  -1.50390625e-01
   4.78515625e-02   7.66601562e-02   2.22656250e-01  -1.48437500e-01
  -1.20849609e-02  -1.40625000e-01  -4.07714844e-02   7.27539062e-02
   4.57763672e-03   3.03955078e-02   1.44531250e-01   1.35742188e-01
  -3.32031250e-02  -1.74804688e-01  -6.64062500e-02   1.56250000e-01
   1.55273438e-01   8.05664062e-03   1.12792969e-01   8.54492188e-03
   1.59179688e-01  -5.12695312e-02   1.66992188e-01  -8.2519531

(None, None)

In [27]:
def pack_data_to_format():
    predictions = joblib.load('dl_predictions.pkl')
    
    test_json = json.load(open('Microblogs_Testdata.json', 'r'))
    # print(test_json[:5])
    n=len(test_json)
    pred_list = []
    for i in range(n):
        data[i] = None

    for i in range(len(test_json)):
        data = {'id': test_json[i]['id'], 'cashtag': test_json[i]['cashtag'], 'sentiment score': str(predictions[i][0])}
        pred_list.append(data)
  

    json.dump(pred_list, open('dl_submission2.json', 'w'))

In [23]:
import json

In [29]:
def get_features(mode):
    loaded_feature_list = []

    for feature_name in config.features_to_use_mb:
        if mode != 'mb' and (feature_name == 'cashtag' or feature_name == 'source'):
            continue
        print('\n---------------------------------------------')
        print( 'Loading {} from '.format(feature_name), end=' ' )

        # Microblog
        if mode == 'mb':
            filename =  'mb_' + feature_name + '.pkl'
        else:
            filename = 'hl_' + feature_name + '.pkl'
        print(filename)

        loaded_feature = joblib.load( filename )

        if not isinstance(loaded_feature, np.ndarray):
            loaded_feature = loaded_feature.toarray()
        print('Shape =  {}, type = {}'.format(loaded_feature.shape, type(loaded_feature)))
        loaded_feature_list.append( loaded_feature )
        print('---------------------------------------------')

    return combine_features(loaded_feature_list)

In [ ]:
pack_data_to_format()


In [ ]:
def main():

    #----------------------------------------------------------------
    # Headline
    MOOD = 'mb'

    print('Loading X')
    X = joblib.load(config.DUMPED_VECTOR_DIR+'mb_sequences.pkl')
    features = get_features(MOOD)
    print(X.shape, features.shape)
    print('Loading Y')

    if MOOD == 'hl':
        y = joblib.load( 'headline_scores.pkl' )
    else:
        y = joblib.load('mb_scores.pkl')
    print(type(X), type(y))
    print(len(y))

    # Microblog Split
    X_train, X_dev, X_test, Y_train, Y_dev = X[:1693], X, X[1693:], y[:1693], y
    features_train, features_dev, features_test = features[:1693], features, features[1693:]
    # ----------------------------------------------------------------
    print(len(X_train), len(X_test), len(X_dev), len(Y_train), len(Y_dev))
    print(len(y))
    deep_learning_cv(X_train, Y_train, 10, X_dev, Y_dev, X_test, features_train, features_dev, features_test)
    # final_predict(X_train, Y_train, X_test, features_train, features_dev, features_test)
    print('MICROBLOG')

In [ ]:
def deep_learning_cv(X_train, y_train, k, X_dev, y_dev, X_test, company_train, company_dev, company_test):
    global X_DIM, Y_DIM
    X_DIM = X_train[0].shape[0]
    y_train = np.array(y_train)
    print(X_train.shape, y_train.shape)
    Y_DIM = 1
    y_test = np.array(y_dev).reshape(1,-1)

    kfold_cv = KFold(n_splits=k, shuffle=True, random_state=7)
    fold_counter = 1
    cos_train = []
    cos_test = []
    cos_trial = []

    # -------------------------------------------------
    # SVR
    # -------------------------------------------------
    # param_grid = {'C': [1000, 100, 10, 1, 0.1, 0.01, 0.001, 0.00001, 0.000001, 0.0000001]}
    #
    # regressor = LinearSVR()
    # best_regressor = GridSearchCV(regressor, param_grid, cv=k, verbose=0, n_jobs=-1,
    #                               scoring=make_scorer(cosine_similarity))
    # best_regressor.fit(company_train, y_train)
    # print("best parameter: ", best_regressor.best_params_)
    # print("best score: ", best_regressor.best_score_)
    # print(best_regressor.grid_scores_)


    # print('LSTM, Epoch', NB_EPOCH)
    print('Fold\t Training\t \t Test \t\t\t\t Trial')
    print('------------------------------------------------------------------------------------')
    for train_index, test_index in kfold_cv.split(X_train):
        print('Size', len(train_index), len(test_index))
        # print('Running Fold {}'.format(fold_counter))
        # print('------------------------------------------')
        print('Fold {}'.format(fold_counter), end='\t')

        X_tr, X_ts = X_train[train_index], X_train[test_index]
        y_tr, y_ts = y_train[train_index], y_train[test_index]
        C_tr, C_ts = company_train[train_index], company_train[test_index]

        callbacks = [
            EarlyStopping(monitor='val_loss', patience=20, verbose=0),
        ]
        # regressor = KerasRegressor(build_fn=attention_imp_merge_exp, nb_epoch=NB_EPOCH, batch_size=BATCH_SIZE, verbose=1)
        regressor = linear_model.ElasticNet(random_state=7)
        # regressor = LinearSVR(C=0.1)
        # print('Training Model')
        if MERGE == True:
            # regressor.fit([X_tr, C_tr], y_tr, callbacks=callbacks)
            regressor.fit([X_tr, C_tr, X_tr], y_tr)
        else:
            print('NOT MERGE', C_tr.shape, y_tr.shape)
            regressor.fit(C_tr, y_tr)
            # regressor.fit(X_tr, y_tr)

        # print('Predicting Tests')
        if MERGE == True:
            # predictions = regressor.predict([X_ts, C_ts])
            # predictions = np.array(predictions).reshape(1, -1)
            # train_predictions = regressor.predict([X_tr, C_tr])
            # trial_predictions = regressor.predict([X_dev, company_dev])
            predictions = regressor.predict([X_ts, C_ts, X_ts])
            predictions = np.array(predictions).reshape(1,-1)
            train_predictions = regressor.predict([X_tr, C_tr, X_tr])
            trial_predictions = regressor.predict([X_dev, company_dev, X_dev])
        else:
            predictions = regressor.predict(C_ts)
            # predictions = regressor.predict(X_ts)
            # test_predictions = regressor.predict(X_dev)
            predictions = np.array(predictions).reshape(1, -1)
            train_predictions = regressor.predict(C_tr)
            # trial_predictions = regressor.predict(company_test)

        cos_train.append( cosine_similarity(y_tr, train_predictions)[0][0] )
        cos_test.append(cosine_similarity(y_ts, predictions)[0][0])
        # cos_trial.append(cosine_similarity(y_test, trial_predictions)[0][0])

        print('{}\t{}'.format(cos_train[-1], cos_test[-1]))
        # print('{}\t{}\t{}'.format(cos_train[-1], cos_test[-1], cos_trial[-1]))
        # print("Cosine Similarity for Training fold ", cos_train[-1])
        # print("Cosine Similarity for Test fold ", cos_test[-1])
        # print("Cosine Similarity for Trial Data ", cos_trial[-1])

        fold_counter += 1

    print('---------------')
    print('Final Result')
    print('Training fold cosine mean : ', np.mean(cos_train))
    print('Testing fold cosine mean : ', np.mean(cos_test))
    print('Trial fold cosine mean : ', np.mean(cos_trial))
    print('{},{},{}'.format(np.mean(cos_train), np.mean(cos_test), np.mean(cos_trial)))
